# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
tf.__version__

'2.5.0-rc0'

## Part 1 - Data Preprocessing

Check out this tutorial on [image data augmentation](https://stepup.ai/exploring_data_augmentation_keras/).

### Preprocessing the Training set

[keras image preprocessing API](https://keras.io/api/preprocessing/image/)
 
We use the [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class) which can generate batches of data with real time transformations, as defined in the construction of the generator object.  

The rescale parameter set to this specific value (1/255) provides [feature scaling](https://en.wikipedia.org/wiki/Feature_scaling): normalization of the input.  
Since each pixel's value is in the range [0 - 255], dividing by 255 will map the values to the [0 - 1] range.

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# target_size: the size of the images when loaded
# here we resize the images to 64x64 pixels
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

When loading the test set, we don't perform any transformations, since we want to predict the class of the actual images. (That's why they're called test images. It wouldn't make much sense trying to predict the classes of different images, which is what we obtain after the augmentation process).  

It is crucial, however, to perform feature scaling, since we've done so when we processed the training images.  
Skipping this step would result in trying to predict image classes for images that are entirely on a different value scale, and the accuracy would be, as a result, very low.

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)

# class_mode: binary/categorical
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [0]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [0]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [0]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [0]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [0]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [0]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [0]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [0]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

## Part 4 - Making a single prediction

In [0]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [0]:
print(prediction)